This first part is only needed as we have to mock VASP here as we cannot run it directly in a jupyter notebook:

In [ ]:
from mock_vasp import TEST_DIR, mock_vasp

ref_paths = {
    "tight relax 1": "Si_gruneisen_1/tight_relax_1",
    "tight relax 2": "Si_gruneisen_1/tight_relax_2",
    "tight relax 1 plus": "Si_gruneisen_1/tight_relax_plus",
    "tight relax 2 plus": "Si_gruneisen_1/tight_relax_plus_2",
    "tight relax 1 minus": "Si_gruneisen_1/tight_relax_minus",
    "tight relax 2 minus": "Si_gruneisen_1/tight_relax_minus_2",
    "phonon static 1/1 ground": "Si_gruneisen_1/phonon_ground",
    "phonon static 1/1 plus": "Si_gruneisen_1/phonon_plus",
    "phonon static 1/1 minus": "Si_gruneisen_1/phonon_minus",
}

# Grüneisen Workflow Tutorial with VASP

## Background
The Grüneisen workflow is based on the implementation in Phonopy.

If you want to read more about Phonopy, please read Togo’s paper: https://doi.org/10.7566/JPSJ.92.012001

## Let's run the workflow
Now, we load a structure and other important functions and classes for running the Grüneisen workflow.

In [ ]:
from jobflow import JobStore, run_locally
from maggma.stores import MemoryStore
from pymatgen.core import Structure

from atomate2.vasp.flows.gruneisen import GruneisenMaker, PhononMaker

job_store = JobStore(MemoryStore(), additional_stores={"data": MemoryStore()})
si_structure = Structure.from_file(TEST_DIR / "structures" / "Si_diamond.cif")

Then one can use the `GruneisenMaker` to generate a `Flow`.

In [ ]:
flow = GruneisenMaker(
    symprec=1e-4,
    phonon_maker=PhononMaker(
        create_thermal_displacements=False,
        store_force_constants=False,
        prefer_90_degrees=False,
        min_length=10,
        born_maker=None,
        bulk_relax_maker=None,
        static_energy_maker=None,
    ),
).make(structure=si_structure)

The Grüneisen parameter workflow will perform 3 different phonon runs at 3 different volumes at and around the equilibrium to compute the mode Grüneisen values.

In [ ]:
flow.draw_graph().show()

We now run the flow with  `run_locally`. We mock the run here. Normally, you would simply use `run_locally` without the `with mock_vasp`

In [ ]:
with mock_vasp(ref_paths=ref_paths) as mf:
    run_locally(
        flow,
        create_folders=True,
        ensure_success=True,
        raise_immediately=True,
        store=job_store,
    )

In [ ]:
job_store.connect()

result = job_store.query_one(
    {"name": "compute_gruneisen_param"},
    properties=[
        "output.gruneisen_band_structure",
        "output.gruneisen_parameter",
    ],
    load=True,
    sort={"completed_at": -1},  # to get the latest computation
)

In [ ]:
from pymatgen.phonon.gruneisen import GruneisenPhononBandStructureSymmLine
from pymatgen.phonon.plotter import GruneisenPhononBSPlotter

bs = GruneisenPhononBandStructureSymmLine.from_dict(
    result["output"]["gruneisen_band_structure"]
)
plt = GruneisenPhononBSPlotter(bs=bs)
plt.get_plot(ylim=[-2, 2])